In [181]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from pathlib import Path

## Importation des données d'entraînement et de test

In [182]:
DATA_PATH = Path("../input/preprocessed")
X_tr = pd.read_csv(DATA_PATH / "X_train.csv", index_col=0,parse_dates=True)
X_test = pd.read_csv(DATA_PATH / "X_test.csv", index_col=0,parse_dates=True)
Y_tr = pd.read_csv(DATA_PATH / "y_train.csv", index_col=0,parse_dates=True)

In [183]:
X_tr = np.expm1(X_tr)
X_test = np.expm1(X_test)
Y_tr = np.expm1(Y_tr)

/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in expm1
  result = func(self.values, **kwargs)


In [184]:
holed_cols = [c for c in X_tr.columns if c.startswith("holed")]
clean_cols = [c for c in X_tr.columns if c not in holed_cols]

In [185]:
assert list(holed_cols) == list(Y_tr.columns)

## Config

In [186]:
class Config:
    hidden_size = 256
    num_layers = 3
    dropout = 0.2
    batch_size = 64
    learning_rate = 0.001
    num_epochs = 50
    patience = 10
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
print(f"Using device: {config.device}")

Using device: cuda


## Classes

## Fonctions d'entraînement

In [187]:
# ============= Dataset =============

class TimeSeriesDataset(Dataset):
    """Dataset pour les séries temporelles avec valeurs manquantes"""
    
    def __init__(self, X, y=None, scaler=None, fit_scaler=False):
        """
        X: DataFrame (timestamps × courbes)
        y: DataFrame avec les vraies valeurs (même format que X)
        """
        self.X = X.values.T  # Shape: (n_series, n_timesteps)
        self.y = y.values.T if y is not None else None
        
        # Normalisation
        if fit_scaler:
            self.scaler = StandardScaler()
            # Fit sur les valeurs non-NaN uniquement
            valid_values = self.X[~np.isnan(self.X)].reshape(-1, 1)
            self.scaler.fit(valid_values)
        else:
            self.scaler = scaler
            
        # Normaliser X
        self.X_scaled = self.X.copy()
        mask = ~np.isnan(self.X)
        self.X_scaled[mask] = self.scaler.transform(self.X[mask].reshape(-1, 1)).flatten()
        
        # Remplacer les NaN par 0 pour le modèle (on utilisera le mask)
        self.X_scaled = np.nan_to_num(self.X_scaled, nan=0.0)
        
        # Normaliser Y aussi !
        if self.y is not None:
            self.y_scaled = self.scaler.transform(self.y.reshape(-1, 1)).reshape(self.y.shape)
        else:
            self.y_scaled = None
        
        # Créer le masque (1 = valeur observée, 0 = manquante)
        self.mask = (~np.isnan(self.X)).astype(np.float32)

    def __len__(self):
        return self.X.shape[0]  # Nombre de séries
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.X_scaled[idx]).unsqueeze(-1)  # (timesteps, 1)
        mask = torch.FloatTensor(self.mask[idx]).unsqueeze(-1)   # (timesteps, 1)
        
        if self.y_scaled is not None:
            y = torch.FloatTensor(self.y_scaled[idx]).unsqueeze(-1)
            return x, mask, y
        return x, mask

# ============= Modèle BiLSTM =============

class BiLSTMImputer(nn.Module):
    """Modèle BiLSTM bidirectionnel pour imputation"""
    
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, dropout=0.2):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # BiLSTM
        self.bilstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True,
            batch_first=True
        )
        
        # Couche de sortie
        self.fc = nn.Linear(hidden_size * 2, 1)  # *2 car bidirectionnel
        
    def forward(self, x, mask):
        """
        x: (batch, timesteps, 1)
        mask: (batch, timesteps, 1)
        """
        # Concaténer x et mask comme input
        x_masked = torch.cat([x, mask], dim=-1)  # (batch, timesteps, 2)
        
        # Passer par le BiLSTM
        lstm_out, _ = self.bilstm(x_masked)  # (batch, timesteps, hidden*2)
        
        # Prédiction
        output = self.fc(lstm_out)  # (batch, timesteps, 1)
        
        return output


# ============= Loss personnalisée =============

class MaskedMSELoss(nn.Module):
    """MSE loss qui ne calcule l'erreur que sur les valeurs manquantes"""
    
    def forward(self, pred, target, mask):
        """
        pred: prédictions (batch, timesteps, 1)
        target: vraies valeurs (batch, timesteps, 1)
        mask: masque (batch, timesteps, 1) - 1=observé, 0=manquant
        """
        # On veut prédire uniquement les valeurs manquantes
        missing_mask = (1 - mask)  # Inverser le masque
        
        # Calculer l'erreur uniquement sur les valeurs manquantes
        error = (pred - target) ** 2
        masked_error = error * missing_mask
        
        # Moyenne sur les valeurs manquantes uniquement
        n_missing = missing_mask.sum()
        if n_missing > 0:
            loss = masked_error.sum() / n_missing
        else:
            loss = torch.tensor(0.0, device=pred.device)
        
        return loss


In [188]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for x, mask, y in tqdm(loader, desc="Training", leave=False):
        x, mask, y = x.to(device), mask.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        # Forward
        pred = model(x, mask)
        loss = criterion(pred, y, mask)
        
        # Backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def validate(model, loader, criterion, device, scaler):
    model.eval()
    total_loss = 0
    total_mae = 0
    
    with torch.no_grad():
        for x, mask, y in loader:
            x, mask, y = x.to(device), mask.to(device), y.to(device)
            
            pred = model(x, mask)
            loss = criterion(pred, y, mask)
            
            # Calculer MAE sur les vraies valeurs (dénormalisées)
            pred_denorm = scaler.inverse_transform(pred.cpu().numpy().reshape(-1, 1))
            y_denorm = scaler.inverse_transform(y.cpu().numpy().reshape(-1, 1))
            mask_cpu = mask.cpu().numpy().reshape(-1, 1)
            
            # MAE seulement sur les valeurs manquantes
            missing_mask = (1 - mask_cpu).astype(bool).flatten()
            if missing_mask.sum() > 0:
                mae = np.abs(pred_denorm.flatten()[missing_mask] - 
                           y_denorm.flatten()[missing_mask]).mean()
                total_mae += mae
            
            total_loss += loss.item()
    
    return total_loss / len(loader), total_mae / len(loader)

def train_model(model, train_loader, val_loader, config, scaler):
    """Entraînement avec early stopping"""
    
    criterion = MaskedMSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(config.num_epochs):
        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion, config.device)
        
        # Validate
        val_loss, val_mae = validate(model, val_loader, criterion, config.device, scaler)
        
        # Scheduler
        scheduler.step(val_loss)
        
        print(f"Epoch {epoch+1}/{config.num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val MAE: {val_mae:.2f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Sauvegarder le meilleur modèle
            torch.save(model.state_dict(), 'best_bilstm_model.pt')
            print("  → New best model saved!")
        else:
            patience_counter += 1
            
        if patience_counter >= config.patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Charger le meilleur modèle
    model.load_state_dict(torch.load('best_bilstm_model.pt'))
    return model

## Fonctions de prédictions

In [189]:
def predict(model, X_df, scaler, config, batch_size=64):
    """Prédire les valeurs manquantes"""
    
    # Créer dataset
    dataset = TimeSeriesDataset(X_df, scaler=scaler, fit_scaler=False)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    all_predictions = []
    
    with torch.no_grad():
        for x, mask in tqdm(loader, desc="Predicting"):
            x, mask = x.to(config.device), mask.to(config.device)
            
            pred = model(x, mask)
            all_predictions.append(pred.cpu().numpy())
    
    # Concatener et reshape
    predictions = np.concatenate(all_predictions, axis=0)  # (n_series, timesteps, 1)
    predictions = predictions.squeeze(-1).T  # (timesteps, n_series)
    
    # Dénormaliser
    predictions_denorm = scaler.inverse_transform(predictions.reshape(-1, 1))
    predictions_denorm = predictions_denorm.reshape(predictions.shape)
    
    # Créer DataFrame
    pred_df = pd.DataFrame(
        predictions_denorm,
        index=X_df.index,
        columns=X_df.columns
    )
    
    # Remplir uniquement les valeurs manquantes
    result = X_df.copy()
    mask = X_df.isna()
    result[mask] = pred_df[mask]
    
    return result

# ============= Fonction principale =============
def run_bilstm_imputation(X_tr, Y_tr, holed_cols, clean_cols, config):
    """
    Pipeline complet d'entraînement et prédiction
    
    Returns:
        model: modèle entraîné
        scaler: scaler utilisé
        train_loader: pour analyse
        val_loader: pour analyse
    """
    
    # Séparer train/val (80/20 sur les courbes holed)
    n_holed = len(holed_cols)
    n_train = int(0.8 * n_holed)
    
    train_cols = holed_cols[:n_train]
    val_cols = holed_cols[n_train:]
    
    print(f"Training on {len(train_cols)} series, validating on {len(val_cols)} series")
    
    # Créer datasets
    train_dataset = TimeSeriesDataset(
        X_tr[train_cols], 
        Y_tr[train_cols], 
        fit_scaler=True
    )
    
    val_dataset = TimeSeriesDataset(
        X_tr[val_cols], 
        Y_tr[val_cols], 
        scaler=train_dataset.scaler
    )
    
    # DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.batch_size, 
        shuffle=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config.batch_size, 
        shuffle=False
    )
    
    # Créer modèle
    model = BiLSTMImputer(
        input_size=2,  # valeur + mask
        hidden_size=config.hidden_size,
        num_layers=config.num_layers,
        dropout=config.dropout
    ).to(config.device)
    
    print(f"\nModel architecture:")
    print(model)
    print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Entraîner
    print("\nStarting training...")
    model = train_model(model, train_loader, val_loader, config, train_dataset.scaler)
    
    return model, train_dataset.scaler, train_loader, val_loader

## Pré-entraînement

In [190]:
# ============= PRÉ-ENTRAÎNEMENT =============

def create_masked_data(X_clean, mask_prob=0.15, seed=42):
    """
    Créer des données synthétiques avec masques aléatoires
    pour le pré-entraînement
    
    Args:
        X_clean: DataFrame avec courbes complètes
        mask_prob: probabilité de masquer une valeur (15% par défaut)
        seed: graine aléatoire
    
    Returns:
        X_masked: DataFrame avec valeurs masquées (NaN)
        Y_true: DataFrame avec vraies valeurs (pour la supervision)
    """
    np.random.seed(seed)
    
    X_masked = X_clean.copy()
    Y_true = X_clean.copy()
    
    print(f"Création de données masquées (mask_prob={mask_prob})...")
    
    # Pour chaque courbe, masquer aléatoirement des timesteps
    for col in tqdm(X_masked.columns, desc="Masking"):
        # Nombre de valeurs à masquer
        n_values = len(X_masked)
        n_to_mask = int(n_values * mask_prob)
        
        # Sélectionner aléatoirement les indices à masquer
        mask_indices = np.random.choice(n_values, size=n_to_mask, replace=False)
        
        # Masquer
        X_masked.loc[X_masked.index[mask_indices], col] = np.nan
    
    # Statistiques
    total_values = X_clean.shape[0] * X_clean.shape[1]
    masked_values = X_masked.isna().sum().sum()
    print(f"✓ Masqué {masked_values:,} valeurs sur {total_values:,} ({masked_values/total_values*100:.1f}%)")
    
    return X_masked, Y_true


def pretrain_model(model, X_tr, clean_cols, config, n_epochs_pretrain=20):
    """
    Pré-entraîner le modèle sur les courbes complètes
    
    Args:
        model: BiLSTMImputer non entraîné
        X_tr: DataFrame complet
        clean_cols: colonnes des courbes complètes
        config: Config object
        n_epochs_pretrain: nombre d'epochs de pré-entraînement
    
    Returns:
        model: modèle pré-entraîné
        scaler: scaler utilisé
    """
    print("\n" + "="*60)
    print("PHASE 1 : PRÉ-ENTRAÎNEMENT sur les 20k courbes complètes")
    print("="*60)
    
    # Créer données masquées
    X_masked, Y_true = create_masked_data(
        X_tr[clean_cols], 
        mask_prob=0.15,  # Masquer 15% des valeurs
        seed=42
    )
    
    # Split train/val (90/10 car on a beaucoup de données)
    n_clean = len(clean_cols)
    n_train = int(0.9 * n_clean)
    
    train_cols_pretrain = clean_cols[:n_train]
    val_cols_pretrain = clean_cols[n_train:]
    
    print(f"\nPré-entraînement : {len(train_cols_pretrain)} train, {len(val_cols_pretrain)} val")

    train_dataset = TimeSeriesDataset(
            X_masked[train_cols_pretrain],
            Y_true[train_cols_pretrain],
            fit_scaler=True
        )
        
    val_dataset = TimeSeriesDataset(
            X_masked[val_cols_pretrain],
            Y_true[val_cols_pretrain],
            scaler=train_dataset.scaler
        )
        
    # DataLoaders
    train_loader = DataLoader(
            train_dataset,
            batch_size=config.batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=True
        )
        
    val_loader = DataLoader(
            val_dataset,
            batch_size=config.batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )
    print(f"\nDébut du pré-entraînement ({n_epochs_pretrain} epochs)...")
    
    # Sauvegarder config original
    original_epochs = config.num_epochs
    config.num_epochs = n_epochs_pretrain
    
    model = train_model(model, train_loader, val_loader, config, train_dataset.scaler)
    
    # Restaurer config
    config.num_epochs = original_epochs
    
    print("\n✓ Pré-entraînement terminé !")
    print("Le modèle a appris les patterns généraux de consommation électrique")
    
    return model, train_dataset.scaler


def finetune_model(model, scaler, X_tr, Y_tr, holed_cols, config):
    """
    Fine-tuner le modèle pré-entraîné sur les vraies courbes à trous
    
    Args:
        model: modèle pré-entraîné
        scaler: scaler du pré-entraînement
        X_tr: DataFrame avec courbes à trous
        Y_tr: vraies valeurs
        holed_cols: colonnes des courbes à trous
        config: Config object
    
    Returns:
        model: modèle fine-tuné
    """
    print("\n" + "="*60)
    print("PHASE 2 : FINE-TUNING sur les 1000 courbes à trous réels")
    print("="*60)
    
    # Split train/val
    n_holed = len(holed_cols)
    n_train = int(0.8 * n_holed)
    
    train_cols = holed_cols[:n_train]
    val_cols = holed_cols[n_train:]
    
    print(f"Fine-tuning : {len(train_cols)} train, {len(val_cols)} val")
    
    # Créer datasets (réutiliser le scaler du pré-entraînement!)
    train_dataset = TimeSeriesDataset(
        X_tr[train_cols],
        Y_tr[train_cols],
        scaler=scaler,
        fit_scaler=False  # Important : ne pas refitter le scaler!
    )
    
    val_dataset = TimeSeriesDataset(
        X_tr[val_cols],
        Y_tr[val_cols],
        scaler=scaler,
        fit_scaler=False
    )
    
    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False
    )
    
    # Fine-tuning avec learning rate plus faible
    print(f"\nDébut du fine-tuning avec LR réduit...")

    # Réduire le learning rate pour le fine-tuning
    original_lr = config.learning_rate
    config.learning_rate = original_lr * 0.1  # 10x plus petit
    
    model = train_model(model, train_loader, val_loader, config, scaler)
    
    # Restaurer learning rate
    config.learning_rate = original_lr
    
    print("\n✓ Fine-tuning terminé !")
    
    return model

In [191]:
def train_with_pretraining(X_tr, Y_tr, holed_cols, clean_cols, config, 
                           n_epochs_pretrain=20):
    """
    Pipeline complet : pré-entraînement + fine-tuning
    
    Args:
        X_tr: DataFrame avec toutes les courbes
        Y_tr: vraies valeurs des courbes à trous
        holed_cols: colonnes des courbes à trous
        clean_cols: colonnes des courbes complètes
        config: Config object
        n_epochs_pretrain: epochs de pré-entraînement
    
    Returns:
        model: modèle entraîné
        scaler: scaler utilisé
    """
    print("\n" + "="*60)
    print("ENTRAÎNEMENT AVEC PRÉ-ENTRAÎNEMENT")
    print("="*60)
    print(f"Courbes complètes (pré-entraînement) : {len(clean_cols)}")
    print(f"Courbes à trous (fine-tuning) : {len(holed_cols)}")
    print("="*60)
    
    # Créer le modèle
    model = BiLSTMImputer(
        input_size=2,
        hidden_size=config.hidden_size,
        num_layers=config.num_layers,
        dropout=config.dropout
    ).to(config.device)
    
    print(f"\nModèle créé : {sum(p.numel() for p in model.parameters()):,} paramètres")
    
    # PHASE 1 : Pré-entraînement
    model, scaler = pretrain_model(
        model, 
        X_tr, 
        clean_cols, 
        config,
        n_epochs_pretrain=n_epochs_pretrain
    )
    
    # PHASE 2 : Fine-tuning
    model = finetune_model(
        model,
        scaler,
        X_tr,
        Y_tr,
        holed_cols,
        config
    )
    
    print("\n" + "="*60)
    print("ENTRAÎNEMENT COMPLET TERMINÉ !")
    print("="*60)
    
    return model, scaler


## Entraînement

In [192]:
config = Config()


# Entraîner avec pré-entraînement
model, scaler = train_with_pretraining(
    X_tr=X_tr,
    Y_tr=Y_tr,
    holed_cols=holed_cols,
    clean_cols=clean_cols,
    config=config,
    n_epochs_pretrain=15  # Ajuster selon le temps disponible
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Training on 799 series, validating on 200 series

Model architecture:
BiLSTMImputer(
  (bilstm): LSTM(2, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

Total parameters: 2,109,953

Starting training...


Epoch 1/50
  Train Loss: 0.6692
  Val Loss: 0.4143, Val MAE: 184.99
  → New best model saved!


Epoch 2/50
  Train Loss: 0.4406
  Val Loss: 0.3670, Val MAE: 178.39
  → New best model saved!


Epoch 3/50
  Train Loss: 0.4049
  Val Loss: 0.3018, Val MAE: 181.16
  → New best model saved!


...

Epoch 46/50
  Train Loss: 0.1525
  Val Loss: 0.1423, Val MAE: 89.25


Epoch 47/50
  Train Loss: 0.1543
  Val Loss: 0.1421, Val MAE: 90.03
  → New best model saved!


Epoch 48/50
  Train Loss: 0.1568
  Val Loss: 0.1420, Val MAE: 89.67
  → New best model saved!


Epoch 49/50
  Train Loss: 0.1543
  Val Loss: 0.1424, Val MAE: 89.31


Epoch 50/50
  Train Loss: 0.1568
  Val Loss: 0.1422, Val MAE: 89.67

✓ Fine-tuning terminé !

ENTRAÎNEMENT COMPLET TERMINÉ !


## Prédiction sur le X_test

In [ ]:
X_test_holed_cols = [c for c in X_test.columns if c.startswith("holed")]
X_test_imputed = predict(model, X_test[X_test_holed_cols], scaler, config)

In [ ]:
X_test_imputed

In [196]:
X_test_imputed.shape

(1057, 1000)

In [ ]:
pd.DataFrame(X_test_imputed, 
             index=X_test.index, 
             columns=X_test_holed_cols).to_csv("predictions_bilstm.csv")